In [3]:
%store -r tokens

In [4]:
tokens = tokens

In [5]:
print(tokens)

['#include', '<', 'iostream', '>', 'using', 'namespace', 'std', ';', 'int', 'main', '(', ')', '{', 'int', 'first_number', ',', 'second_number', ',', 'sum', ';', 'char', 'c', '=', "'d'", ';', 'cout', '<<', '"Enter two integers: "', ';', 'cin', '>>', 'first_number', '>>', 'second_number', ';', 'sum', '=', 'first_number', '+', 'second_number', ';', 'if', '(', 'true', ')', '{', 'cout', '<<', '"how are you"', ';', '}', 'cout', '<<', 'first_number', '<<', '" + "', '<<', 'second_number', '<<', '" = "', '<<', 'sum', ';', 'return', '0', ';', '}']


In [6]:
global_symbol_table = {}

# symbol table structure {'a':['int'], 'func1' : ['int', {'a': 'int'}], 'func2' : ['void', {}]}

In [7]:
def ignorePreprocessor(tokens, index):
    while(tokens[index] != ';'):
        index+=1
    return index+1

In [8]:
datatype=["int","double","float","char","string", "void", "bool"]

integer_dtypes = ["int"]
decimal_dtypes = ["float", "double"]
boolean_dtypes = ["bool"]

In [9]:
rval_operators=['+', '-', '*', '/', '&&', '||', '&', '|', '<', '>', '^', '==', '>=', '<=']
lval_operators=['+=', '-=', '=', '*=', '/=']
unary=['!', '-', '+']

In [10]:
def isIdentifier(s):
    if(not((s[0]=='_')or(s[0]>='a' and s[0]<='z')or(s[0]>='A' and s[0]<='Z'))):
        return False
    for i in range(1,len(s)):
        if(not((s[i]=='_')or(s[i]>='a' and s[i]<='z')or(s[i]>='A' and s[i]<='Z')or(s[i]>='0' and s[i]<='9'))):
            return False
    return True

In [11]:
def checkVarHeader(tokens, index):
    # check construct of type <dtype> <ident>
    dtype=""
    ident=""

    if(tokens[index] not in datatype):
        return index, "invalid data type", dtype, ident
    else:
        dtype = tokens[index]
    index += 1

    # check for identifier
    if(not isIdentifier(tokens[index])):
        return index, "invalid identifier name", dtype, ident
    else:
        ident = tokens[index]
        
    return index+1, None, dtype, ident

In [12]:
def checkParametersList(tokens, index):
    parameters = {}
    if(tokens[index] != '('):
        return index, "missing opening parenthesis for a function", parameters
    
    index += 1
    while(tokens[index] != ')'):
        index, err, dtype, ident = checkVarHeader(tokens, index)
        parameters[ident] = dtype
        if(err):
            return index, err, parameters
        if(tokens[index] == ','):
            index += 1
        else:
            break
    
    if(tokens[index] != ')'):
        return index, "missing closing parenthesis for a function", parameters
    
    return index, None, parameters


In [13]:
def checkLval(tokens, index, symbol_table):
    dtype = ""
    if(isIdentifier(tokens[index])):
        # check if present in symbol table and get dtype from symbol table
        ident = tokens[index]

        if(ident not in symbol_table):
            return index, "variable not declared"
        index += 1

        # if the ident is a function
        if(tokens[index] == '('):
            index += 1
            params = []
            while(tokens[index] != ')'):
                if(not isIdentifier(tokens[index])):
                    return index, "invalid identifier", dtype, False
                index+=1
                
                id = tokens[index]
                if(id in symbol_table):
                    params.append(symbol_table[id][0])
                if(tokens[index] != ',' and tokens[index] != ')'):
                    return index, "unexpected token", dtype, False
                index += 1
            
            
                if(len(symbol_table[ident]) < 2):
                    return index, "identifier is not callable", dtype, False
                
                params_list = list(symbol_table[ident][1].values())
                if(params_list != params):
                    return index, "parameter list of function doesn't match", dtype, False
                
            return index, None, symbol_table[ident][0], False
   
        # if not a function call
        return index, None, symbol_table[ident], True
        
    if(tokens[index] in unary):
        op = tokens[index]
        index += 1
        index, err, data_type = checkLval(tokens=tokens, index=index, symbol_table=symbol_table)
        if(op == '!'):
            return index, err, 'bool', False
        else: # operator = '-' or '+'
            if((data_type not in integer_dtypes) and (data_type not in decimal_dtypes) or (data_type not in boolean_dtypes)):
                return index, f"operator {op} on non numerical data type"
            return index, err, data_type, False

    if(tokens[index] == '('):
        index += 1
        index, err, data_type = checkLval(tokens=tokens, index=index, symbol_table=symbol_table)
        if(err):
            return index, err, data_type, False
        
        if(tokens[index] != ')'):
            return index, "unmatched braces", data_type, False
        index += 1 # matched braces
        return index, err, data_type, False

    return index, "unexpected token", data_type, False


In [14]:
def checkExpr(tokens, index, symbol_table, islval):

    dtype = ""
    index, err, ldtype, islvalue = checkLval(tokens, index, symbol_table)
    if(err):
        return index, err

    if(tokens[index] == ";"):
        index += 1
        return index, None, ldtype

    if(tokens[index] in lval_operators):
        if(islval and islvalue):
            index += 1
            index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=True)
        else:
            return index, "lval operators cannot be applied on rval", dtype
    
    elif(tokens[index] in rval_operators):
        index += 1
        index, err, rdtype = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table, islval=False)

    if(err): # error already then return error
        return index, err

    if(tokens[index] == ";"):
        index += 1
        # decide dtype on basis of ldtype and rdtype
        if(ldtype == "void" or rdtype == "void"):
            return index, "no operation allowed on void datatype"

        if(ldtype == "string"):
            if(not (rdtype == "char" or rdtype == "string")):
                return index, "mismatched types", dtype
            return index, None, "string"
        
        if(ldtype == "char"):
            if(rdtype == "string"):
                return index, "string cannot be appended to a char", dtype
            
            if(rdtype in boolean_dtypes):
                return index, None, "int"

            return index, None, rdtype

        if(ldtype in integer_dtypes):

            if(rdtype == "string"):
                return index, "mismatched types", dtype

            if(rdtype == "char"):
                return index, None, ldtype

            if(rdtype in boolean_dtypes):
                return index, None, ldtype

            if(rdtype in decimal_dtypes):
                return index, None, rdtype

            if(rdtype in integer_dtypes):
                if(integer_dtypes.index(rdtype) >= integer_dtypes.index(ldtype)):
                    return index,None, rdtype
                else:
                    return index,None, ldtype

        if(ldtype in decimal_dtypes):

            if(rdtype == "string"):
                return index, "mismatched types", dtype

            if(rdtype == "char"):
                return index, None, ldtype

            if(rdtype in boolean_dtypes):
                return index, None, ldtype

            if(rdtype in integer_dtypes):
                return index, None, ldtype

            if(rdtype in decimal_dtypes):
                if(decimal_dtypes.index(rdtype) >= decimal_dtypes.index(ldtype)):
                    return index,None, rdtype
                else:
                    return index,None, ldtype

        if(ldtype in boolean_dtypes):
            if(rdtype == "string"):
                return index, "mismatched types", dtype

            if(rdtype == "char"):
                return index, None, "int"
            
            return index, None, rdtype

    return index, "unexpected token", dtype


In [15]:
def checkFuncDecOrValDef(tokens, index):
    if(tokens[index] in datatype):
        index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err

    global_symbol_table[ident] = dtype

    
    # variable decl
    if(tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        return index+1, None
    
        # variable def
    if(tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        index += 1
        return checkExpr(tokens, index, global_symbol_table)

    index, err, parameters = checkParametersList(tokens=tokens, index=index)

    if(err):
        return index, err
    
    # add function to the symbol table
    global_symbol_table[ident] = [dtype, parameters]

    if(tokens[index] != ";"):
        return index, "Invalid function declaration"
    
    return index+1, None

In [16]:
def checkScope(tokens, index, symbol_table):
    if(tokens[index] != '{'):
        return index, f"Expected {{ but found {tokens[index]}"

    while(tokens[index] != "}"):

        if(tokens[index] in datatype):
            index, err = checkFuncDecOrValDef(tokens=tokens, index=index)
            if(err):
                return index, err
            continue
            
        
        elif(tokens[index] == "if"):
            index += 1
            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            
            index, err = checkExpr(tokens=tokens, index=index)
            if(err):
                return index, err
            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"

            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err

            if(tokens[index] == "else"):
                index += 1
                index, err = checkScope(tokens=tokens, index=index, symbol_table=symbol_table)
                if(err):
                    return index, err
            continue

        elif(tokens[index] == "while"):
            index += 1

            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index+=1
            
            index, err = checkExpr(tokens=tokens, index=index)
            if(err):
                return index, err

            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=symbol_table)

            if(err):
                return index, err
            continue

        elif(tokens[index] == "do"):
            index += 1

            index, err = checkScope(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err

            if(tokens[index] != "while"):
                return index, f"Expected while but found {tokens[index]}"
            index += 1

            if(tokens[index] != "("):
                return index, f"Expected ( but found {tokens[index]}"
            index += 1

            index, err = checkExpr(tokens=tokens, index=index)
            if(err):
                return index, err

            if(tokens[index] != ")"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            if(tokens[index] != ";"):
                return index, f"Expected ) but found {tokens[index]}"
            index +=1

            continue

        else:
            index, err = checkExpr(tokens=tokens, index=index, symbol_table=symbol_table)
            if(err):
                return index, err

            continue
        

In [17]:
def checkFuncDefOrVarDef(tokens, index):
    index, err, dtype, ident = checkVarHeader(tokens=tokens, index=index)

    if(err):
        return index, err
    
    global_symbol_table[ident] = dtype
    
    # variable decl
    if(tokens[index] == ";"):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        return index+1, None
    
    # variable def
    if(tokens[index] == "="):
        if(dtype == "void"):
            return index, "wrong datatype for variable declation"
        index += 1
        return checkExpr(tokens, index, global_symbol_table)

    index, err, parameters = checkParametersList(tokens=tokens, index=index)

    if(err):
        return index, err

    # add function to the symbol table
    global_symbol_table[ident] = [dtype, parameters]

    if(tokens[index] == ";"):
        return index+1, None
    
    local_symbol_table = {}
    local_symbol_table.update(global_symbol_table)
    local_symbol_table.update(parameters)

    index, err = checkScope(tokens, index, local_symbol_table)

    return index+1, None
    

In [18]:
i = 0
while(i < len(tokens)):
    
    if(tokens[i][0] == '#'):
        i = ignorePreprocessor(tokens=tokens, index=i)
        continue

    if(tokens[i] in datatype):
        # Def includes Decl, Def
        i, err = checkFuncDefOrVarDef(tokens=tokens, index = i)
        if(err):
            print(f"Error: error occured at token {i, tokens[i]}: {err}")
            break
        continue

KeyboardInterrupt: 